# Detoxify

In [1]:
# add project root to PYTHONPATH
import sys
sys.path.append("../../")

In [2]:
# dependencies
import os

import nltk
import detoxify
import profanity_check
from tqdm import tqdm
import pandas as pd

import constants as cst

In [3]:
def sublists(l, max_len=None):
    for i in range(len(l)):
        if max_len:
            assert isinstance(max_len, int)
            for j in range(i, min(i + max_len, len(l))):
                yield l[i:j + 1]
        else:
            for j in range(i, len(l)):
                yield l[i:j + 1]

def detox_score(texts, model, key):
    return model.predict(texts)[key]


def profanity_score(texts):
    return profanity_check.predict_prob(texts)

In [4]:

model = detoxify.Detoxify()

In [27]:
def find_toxic_phrases(text, l_limit=None, u_limit=None, threshhold=0.4):
    sents = nltk.tokenize.sent_tokenize(text)
    outputs = []
    for sent in sents[l_limit:u_limit]:
        spit_string = sent.split(" ")
        substrings = [" ".join(sublist) for sublist in sublists(spit_string)]
        probs = detox_score(substrings, model, "toxicity")
        probs = [p + 0.003 * (len(s) ** 0.5) for s, p in zip(substrings, probs)]
        for prob, string in zip(probs, substrings):
            if prob > threshhold:
                outputs.append((prob, string))

    return outputs

In [28]:
discord_data_path = cst.PROJECT_ROOT + "/data/raw/ext/discord/"
discord_chats = os.listdir(discord_data_path)
data = pd.read_csv(discord_data_path + discord_chats[0])

import random
chat_strings = [s for s in data["Content"].to_list() if isinstance(s, str)]
for s in chat_strings[-500:-400]:
    output = find_toxic_phrases(s, threshhold=0.2)
    if output:
        print(max(output, key=lambda t: t[0]) + (s,))
    # for t in output:
    #     print(t + (s,))

(1.0062411068938004, 'fuck you', 'fuck you')
(0.9198547568321228, 'too trash', 'this laptop too trash')
(0.34658632553040136, ' come back', '@neo151219  come back')
(0.7230622373606327, 'want some bgm', 'want some bgm')
(0.5053926943800674, 'My fking', 'My fking bad')
(0.3640147430637284, 'you', '@disappointed_doritos where have you been')
(1.0048799899858152, 'poor pussy', 'poor pussy')
(0.364014713261406, 'you', 'you have been taking care of your cat instead of playing _League of Legends_?')
(0.8494009489712859, 'what a poor', 'what a poor allocation of time')
(0.9130074373690839, 'trash', 'trash game')
(0.3640147430637284, 'you', '@Jonathan Cheung are you coming')
(0.364014713261406, 'you', 'I invite all of you to join me')
(0.3640147430637284, 'you', 'To witness greatness you know')
(0.364014713261406, 'you', 'it is a sad day for you')
(0.9820740810839886, 'penis', 'penis')
(0.9300549267790542, 'They die', 'They die')
